In [7]:
import os
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, LSTM
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils


# Define train and test paths
train_path = 'C:/Users/Nayan/Desktop/Project/DevanagariHandwrittenDigitDataset/Train'
test_path = 'C:/Users/Nayan/Desktop/Project/DevanagariHandwrittenDigitDataset/Test'

# Define the number of classes
num_classes = 10

# Define image size and number of channels
img_rows, img_cols, img_channels = 32, 32, 1

# Define a function to load the data from the specified path
import re

def load_data(path):
    data = []
    labels = []
    pattern = r"\d+" # regular expression to match digits in folder name
    for folder in os.listdir(path):
        folder_path = os.path.join(path, folder)
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, (img_rows, img_cols))
            data.append(image)
            match = re.search(pattern, folder)
            labels.append(int(match.group())) # convert matched digits to integer label
    data = np.array(data)
    data = data.reshape(data.shape[0], img_rows, img_cols, img_channels)
    data = data.astype('float32')
    data /= 255
    labels = np.array(labels)
    labels = np_utils.to_categorical(labels, num_classes)
    return data, labels

# Load the training data
X_train, y_train = load_data(train_path)

# Load the testing data
X_test, y_test = load_data(test_path)

# Define the CNN-LSTM model
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(img_rows, img_cols, img_channels)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
from keras.layers import Reshape
from keras.layers import TimeDistributed, LSTM, Dense
model.add(Flatten())
model.add(Reshape((8, 8, 64)))

model.add(TimeDistributed(Flatten()))
model.add(TimeDistributed(Dense(128)))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))





# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])




In [8]:
# Fit the model to the training data
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=64)



Epoch 1/20
266/266 [==============================] - 23s 68ms/step - loss: 0.5246 - accuracy: 0.8169 - val_loss: 0.1126 - val_accuracy: 0.9657
Epoch 2/20
266/266 [==============================] - 17s 64ms/step - loss: 0.0864 - accuracy: 0.9733 - val_loss: 0.0663 - val_accuracy: 0.9810
Epoch 3/20
266/266 [==============================] - 17s 65ms/step - loss: 0.0481 - accuracy: 0.9851 - val_loss: 0.0829 - val_accuracy: 0.9717
Epoch 4/20
266/266 [==============================] - 17s 65ms/step - loss: 0.0298 - accuracy: 0.9914 - val_loss: 0.0369 - val_accuracy: 0.9880
Epoch 5/20
266/266 [==============================] - 17s 65ms/step - loss: 0.0225 - accuracy: 0.9931 - val_loss: 0.0390 - val_accuracy: 0.9887
Epoch 6/20
266/266 [==============================] - 17s 65ms/step - loss: 0.0241 - accuracy: 0.9931 - val_loss: 0.0246 - val_accuracy: 0.9913
Epoch 7/20
266/266 [==============================] - 18s 66ms/step - loss: 0.0168 - accuracy: 0.9948 - val_loss: 0.0329 - val_accuracy:

In [9]:

# Evaluate the model on the testing data
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 99.47%
